In [1]:
import pdfplumber
import pandas as pd
import re
from fuzzywuzzy import fuzz, process
import tkinter as tk

class DiseaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Disease Information Finder")
        self.root.geometry("900x700")
        self.is_dark_mode = False
        
        self.canvas = tk.Canvas(root)
        self.scrollbar = tk.Scrollbar(root, orient="vertical", command=self.canvas.yview)
        self.canvas.config(yscrollcommand=self.scrollbar.set)

        self.chat_frame = tk.Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.chat_frame, anchor="nw")
        self.scrollbar.pack(side="right", fill="y")
        self.canvas.pack(side="top", fill="both", expand=True)

        self.chat_frame.bind("<Configure>", self.on_frame_configure)

        self.input_frame = tk.Frame(root)
        self.input_frame.pack(pady=10, fill="x", padx=20)

        self.label = tk.Label(self.input_frame, text="Enter Cause:", font=("Arial", 12))
        self.label.pack(side="left", padx=10)

        self.entry = tk.Entry(self.input_frame, width=50, font=("Arial", 12))
        self.entry.pack(side="left", padx=10)
        self.entry.bind("<Return>", self.on_enter)

        self.send_button = tk.Button(self.input_frame, text="→", command=self.search, width=5, height=1, font=("Arial", 16, "bold"))
        self.send_button.pack(side="left", padx=10)

        self.pdf_paths = ["up1.pdf","up2.pdf","up3.pdf"]
        self.df = self.extract_data_from_pdfs(self.pdf_paths)

        self.dark_mode_button = tk.Button(root, text="🌙", command=self.toggle_theme, font=("Arial", 12))
        self.dark_mode_button.pack(pady=10, anchor="ne", padx=10)

        self.set_theme()

    def extract_data_from_pdfs(self, pdf_paths):
        data = []
        for pdf_path in pdf_paths:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        for row in table[1:]:  # Assuming the first row is a header
                            if len(row) >= 3:  # Ensure valid data
                                disease = row[0]
                                symptoms = row[1]
                                causes = row[2]
                                precautions = row[3] if len(row) > 3 else ''
                                data.append([disease, symptoms, causes, precautions])

        return pd.DataFrame(data, columns=["Disease", "Symptoms", "Causes", "Precaution"])

    def clean_text(self, text):
        return re.sub(r'\s+', ' ', text).strip().lower() if isinstance(text, str) else ""

    def match_partial_input(self, cause_input, threshold=80):
        cause_input = self.clean_text(cause_input)
        all_causes = self.df['Causes'].tolist()
        best_match, score = process.extractOne(cause_input, all_causes, scorer=fuzz.partial_ratio)

        if score >= threshold:
            result = self.df[self.df['Causes'].apply(lambda x: fuzz.partial_ratio(x, best_match) >= threshold)]
            if len(result) > 1:
                return "This cause is linked to multiple diseases. Please provide more details."

            return "\n".join([
                f"Disease: {row['Disease']}\nSymptoms: {row['Symptoms']}\nCause: {row['Causes']}\nPrecaution: {row['Precaution']}\n{'-'*50}"
                for _, row in result.iterrows()
            ])
        return "No diseases found matching the cause in the database."

    def toggle_theme(self):
        self.is_dark_mode = not self.is_dark_mode
        self.set_theme()

    def set_theme(self):
        self.bg_color = "#2E2E2E" if self.is_dark_mode else "#F0F0F0"
        self.text_color = "#FFFFFF" if self.is_dark_mode else "#000000"
        self.button_color = "#444444" if self.is_dark_mode else "#4CAF50"

        self.root.config(bg=self.bg_color)
        self.chat_frame.config(bg=self.bg_color)
        self.input_frame.config(bg=self.bg_color)
        self.label.config(bg=self.bg_color, fg=self.text_color)
        self.entry.config(bg=self.bg_color, fg=self.text_color)
        self.send_button.config(bg=self.button_color, fg=self.text_color)
        self.dark_mode_button.config(bg=self.button_color, fg=self.text_color)

    def on_enter(self, event=None):
        self.search()

    def search(self):
        cause_input = self.entry.get().strip()
        if not cause_input:
            self.display_message("Please enter a cause to search.", "error")
            return
        
        self.display_message(f"You: {cause_input}", "user")
        result = self.match_partial_input(cause_input)

        self.display_message(result, "bot" if "No diseases found" not in result else "error")
        self.entry.delete(0, tk.END)

    def on_frame_configure(self, event=None):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))

    def display_message(self, message, sender="bot"):
        lines = message.split("\n")
        precaution_found = False

        for idx, line in enumerate(lines):
            # Check if we have reached the 'Precaution' line
            if 'precaution' in line.lower():
                precaution_found = True

            # Using after method to simulate line-by-line display with a delay
            # Bold lines after 'Precaution'
            self.root.after(idx * 500, self._create_message_line, line, sender, precaution_found)

            # If precaution is found, all subsequent lines will be bold
            if precaution_found:
                precaution_found = True

    def _create_message_line(self, message, sender="bot", bold=False):
        colors = {"user": "#DCF8C6", "bot": "#FFFFFF", "error": "#F8D7DA"}
        anchor = "e" if sender == "user" else "w"
        
        # Check if bold is enabled
        font_style = ("Arial", 12, "bold") if bold else ("Arial", 12)
        
        message_label = tk.Label(self.chat_frame, text=message, bg=colors[sender], font=font_style,
                                 justify="left", wraplength=500, padx=10, pady=5, anchor=anchor)
        message_label.pack(anchor=anchor, pady=5, padx=20, fill="x", ipadx=10)

if __name__ == "__main__":
    root = tk.Tk()
    app = DiseaseApp(root)
    root.mainloop()
